# Question2

In [1]:
from py2neo import Graph, NodeSelector, Node, Relationship
import pandas as pd

In [2]:
graph = Graph("http://localhost:7474/db/data/")

In [3]:
movie4 = Node("Movie", title="John Wick")
keanu = Node("Actor",name="Keanu Reeves")
william = Node("Actor", name="William Dafoe")
michale = Node("Actor", name="Michael Nyquist")
chad = Node("Actor", name="Chad Stahelski")
david = Node("Actor", name="David Leitch")

In [4]:
graph.create(movie4|keanu|william|michale|chad|david)

In [5]:
keanu_r = Relationship(keanu, "ACTS_IN", movie4, role = "actor")
graph.create(keanu_r )
william_r = Relationship(william, "ACTS_IN", movie4, role = "actor")
graph.create(william_r )
michale_r = Relationship(michale, "ACTS_IN", movie4, role = "actor")
graph.create(michale_r )
chad_r = Relationship(chad, "DIRECTED", movie4, role = "director")
graph.create(chad_r )
david_r = Relationship(david, "DIRECTED", movie4, role = "director")
graph.create(david_r )

In [8]:
options = {"Actor": "name"}
#draw(graph, options)

In [6]:
selector = NodeSelector(graph)
#selected = selector.select("Actor", name="Keanu Reeves")
selected = selector.select("Actor")
list(selected)

[(keanu_reeves:Actor {name:"Keanu Reeves"}),
 (chad_stahelski:Actor {name:"Chad Stahelski"}),
 (william_dafoe:Actor {name:"William Dafoe"}),
 (michael_nyquist:Actor {name:"Michael Nyquist"}),
 (david_leitch:Actor {name:"David Leitch"})]

# Question3

In [7]:
query = """
MATCH (a:Actor)-[:ACTS_IN]->(movie:Movie)<-[:ACTS_IN]-(Actor{name:"Keanu Reeves" })
RETURN a.name
"""
data = graph.run(query)
for d in data:
    print(d)

(u'a.name': u'Michael Nyquist')
(u'a.name': u'William Dafoe')


In [8]:
query = """
MATCH (a:Actor)-[:DIRECTED]->(movie:Movie)<-[:ACTS_IN]-(Actor{name:"Keanu Reeves" })
RETURN a.name
"""
data = graph.run(query)
for d in data:
    print(d)

(u'a.name': u'David Leitch')
(u'a.name': u'Chad Stahelski')


# Question4

In [9]:
query = """
MATCH (p:Actor)-[r:ACTS_IN]->(m:Movie) RETURN p.name,r.role,m.title
union
MATCH (p:Actor)-[r:DIRECTED]->(m:Movie) RETURN p.name,r.role,m.title
"""

data = graph.run(query)
mydata = []
for d in data:
    mydata.append(d)

In [10]:
df = pd.DataFrame(mydata)
df.columns = ['name', 'role','movie']
df

,name,role,movie
0,Michael Nyquist,actor,John Wick
1,William Dafoe,actor,John Wick
2,Keanu Reeves,actor,John Wick
3,David Leitch,director,John Wick
4,Chad Stahelski,director,John Wick


In [11]:
df.to_csv('df.csv',index=False,header=True)

In [71]:
#movie
query = """
MATCH (m:Movie) RETURN m.title
"""

data = graph.run(query)
mydata = []
for d in data:
    mydata.append(d)
df = pd.DataFrame(mydata)
df.columns = ['title']
df.to_csv('df_movie.csv',index=False,header=True)

In [72]:
df

,title
0,John Wick


In [84]:
#create movie node
query = """ 
LOAD CSV WITH HEADERS FROM "file:///df_movie.csv" AS line
CREATE (m:Movie { title:line.title});
"""
graph.run(query)

In [73]:
#Actor
query = """
MATCH (a:Actor) RETURN a.name
"""

data = graph.run(query)
mydata = []
for d in data:
    mydata.append(d)
df = pd.DataFrame(mydata)
df.columns = ['name']
df.to_csv('df_actor.csv',index=False,header=True)
df

,name
0,Keanu Reeves
1,Chad Stahelski
2,Michael Nyquist
3,David Leitch
4,William Dafoe


In [87]:
query = """ 
LOAD CSV WITH HEADERS FROM "file:///df_actor.csv" as line
CREATE (a:Actor { name:line.name});
"""
graph.run(query)

In [74]:
#roles
query = """
MATCH (p:Actor)-[r:ACTS_IN]->(m:Movie) RETURN p.name,m.title,r.role
union
MATCH (p:Actor)-[r:DIRECTED]->(m:Movie) RETURN p.name,m.title,r.role
"""

data = graph.run(query)
mydata = []
for d in data:
    mydata.append(d)
df = pd.DataFrame(mydata)
df.columns = ['name','title','role']
df.to_csv('df_roles.csv',index=False,header=True)
df

,name,title,role
0,Michael Nyquist,John Wick,actor
1,William Dafoe,John Wick,actor
2,Keanu Reeves,John Wick,actor
3,David Leitch,John Wick,director
4,Chad Stahelski,John Wick,director


In [88]:
query = """
LOAD CSV WITH HEADERS FROM "file:///df_roles.csv" AS line 
MATCH (a:Actor { name:line.name })
MATCH (m:Movie { title:line.title })
CREATE (a)-[:ACTED_IN { role: [line.role]}]->(m);
"""
graph.run(query)

In [75]:
#movie_actor_role
query = """
MATCH (p:Actor)-[r:ACTS_IN]->(m:Movie) RETURN m.title, p.name, r.role
union
MATCH (p:Actor)-[r:DIRECTED]->(m:Movie) RETURN m.title, p.name, r.role
"""

data = graph.run(query)
mydata = []
for d in data:
    mydata.append(d)
df = pd.DataFrame(mydata)
df.columns = ['title', 'name','role']
df.to_csv('df_movie_actor_roles.csv',index=False,header=True)
df

,title,name,role
0,John Wick,Michael Nyquist,actor
1,John Wick,William Dafoe,actor
2,John Wick,Keanu Reeves,actor
3,John Wick,David Leitch,director
4,John Wick,Chad Stahelski,director
